# Simulações de Monte Carlo

Um dos casos de uso mais interessantes das distribuições de probabilidade é usá-las para criar simulações. Com as **simulações de Monte Carlo**, podemos modelar fenômenos do mundo real nos quais estamos interessados ​​com eventos aleatórios seguindo distribuições de probabilidade. Podemos ver como diferentes variáveis ​​interagem entre si e quais resultados convergem.

Depois de aprender a extrair valores aleatórios de uma distribuição normal, modelaremos uma simulação do problema de Monty Hall, bem como uma fila de espera de clientes. Por fim, um exercício consolidará a compreensão usando uma simulação de peso de elevador.

## Problema de Monty Hall

Você já deve ter ouvido falar do Problema de Monty Hall, um famoso problema de probabilidade originalmente apresentado no game show *Let's Make a Deal*. O problema estrutura um competidor escolhendo entre 3 portas, onde uma tem o prêmio e as outras duas têm cabras.

img

As coisas ficam interessantes depois que o competidor escolhe uma porta. Digamos que ele escolha a porta nº 1. O apresentador do game show (que sabe qual porta tem o prêmio) abre uma das outras portas para revelar uma cabra.

img

O competidor agora tem a oportunidade de trocar para a outra porta fechada. Então, a questão é: o competidor aumenta sua probabilidade trocando ou permanecendo?

> Como exercício bônus, você pode usar o Teorema de Bayes para descobrir se permanecer ou trocar aumenta a probabilidade de ganhar o prêmio?

Na década de 1990, [uma colunista chamada Marilyn vos Savant](https://priceonomics.com/the-time-everyone-corrected-the-worlds-smartest/) causou um grande rebuliço na comunidade científica. Ela propôs que um competidor dobrasse suas chances ao trocar de probabilidade, passando de uma probabilidade de $ \frac{1}{3} $ para $ \frac{2}{3} $. A comunidade matemática e científica, de forma veemente, pública e cruel, disse a ela que ela estava errada. No entanto, ela foi provada certa.

img

Poderíamos usar o Teorema de Bayes para provar que Marilyn está correta, e eu o encorajei a fazer esse exercício! No entanto, quero me concentrar em simulações de Monte Carlo como um veículo para esse tipo de descoberta. Com nossa simulação "Monte Carlo Monty Hall", mostraremos evidências empíricas de que a troca de jogos de fato dobra a chance de vitória do competidor.

O código Python abaixo gerará 10.000 jogos aleatórios de Monty Hall (especificados pelo `trial_count`). Em cada jogo, ele selecionará aleatoriamente a porta do prêmio e a porta selecionada pelo competidor. A porta aberta será a porta que não foi selecionada nem tem o prêmio. A porta da troca será a porta que não foi selecionada nem foi aberta. Acompanharemos a frequência com que as vitórias ocorrerão ao permanecer em jogo versus alternar.

In [ ]:
from random import randint, choice

def porta_aleatoria(): return randint(1, 3)

count = 10000

ficou_venceu = 0
trocou_venceu = 0

for i in range(0, count):
    porta_premiada = porta_aleatoria()
    porta_escolhida = porta_aleatoria()
    porta_aberta = choice([d for d in range(1, 4) if d != porta_escolhida and d != porta_premiada])
    porta_trocada = choice([d for d in range(1, 4) if d != porta_escolhida and d != porta_aberta])

    if porta_escolhida == porta_premiada:
        ficou_venceu += 1

    if porta_trocada == porta_premiada:
        trocou_venceu += 1

print("PERMANECEU VENCEU: {}, TROCOU VENCEU: {}".format(
    ficou_venceu, trocou_venceu))

print("TAXA DE GANHO DE PERMANÊNCIA: {}, TAXA DE GANHO DE TROCA: {}".format(
    float(ficou_venceu)/float(count), float(trocou_venceu)/float(count)))

Interessante. Se você executar esta simulação repetidamente, verá consistentemente que cerca de 1/3 dos resultados vencem ao permanecer, mas 2/3 dos resultados vencem ao trocar. Esta é uma evidência bastante convincente de que Marilyn está correta. Afinal, podemos argumentar que o jogo Monty Hall é (se for justo) completamente aleatório. Portanto, foi bem fácil modelar isso em um computador.

Mas por que nos importamos? Jogos são uma coisa e são facilmente modelados. Mas e os problemas da vida real? Embora modelos e simulações estejam longe de ser perfeitos para problemas reais mais complexos, eles podem ser úteis. Vamos primeiro aprender como gerar valores aleatórios contínuos a partir de uma distribuição de probabilidade contínua.

## Gerando Valores a partir de Distribuições

Para aproveitar melhor as simulações de Monte Carlo, precisamos aprender a gerar valores aleatórios a partir de distribuições de probabilidade específicas. Com apenas o conhecimento das seções anteriores, você pode gerar valores aleatórios usando uma função de ponto percentual (FPP) e um valor gerado aleatoriamente entre 0 e 1. Ao gerar valores de probabilidade aleatórios entre 0 e 1, podemos consultar esse valor na FPP de uma distribuição de probabilidade e obter valores aleatórios que seguem essa distribuição.

In [ ]:
from scipy.stats import norm
import random
import matplotlib.pyplot as plt

mean, std = 0, 1

X = []
for i in range(100):
    # valor aleatório entre 0 e 1
    random_p = random.random()

    # procurA essa probabilidade aleatória no PPF normal
    random_x = norm.ppf(random_p, mean, std)
    
    X.append(random_x)

plt.plot(X, [0 for _ in X], 'o') # usa um gráfico de dispersão para fazer uma reta numérica

A maneira mais fácil de gerar valores aleatórios a partir de qualquer distribuição é usar o NumPy em vez do SciPy, utilizando as distribuições do pacote random. Abaixo, geramos aleatoriamente 10 valores a partir da distribuição normal, onde `loc` é a média e `scale` é o desvio padrão. Obteremos um vetor de valores que seguem essa distribuição normal.

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt

X = np.random.normal(loc=0, scale=1, size=100)

print(X)
plt.plot(X, [0 for _ in X], 'o') # usa um gráfico de dispersão para fazer uma reta numérica
plt.show()

Usaremos o NumPy no restante desta seção para gerar valores aleatórios para uma distribuição de probabilidade desejada. A seguir, vejamos um caso útil para essa operação.

## Simulação de fila de clientes

Seja na Disneylândia ou no supermercado, ninguém gosta de esperar em filas. Aliás, essa área da pesquisa operacional é tão predominante que até tem nome próprio. A [Teoria das Filas](https://en.wikipedia.org/wiki/Queueing_theory) estuda como otimizar o tempo de espera das pessoas na fila. Muitas empresas estudaram, otimizaram e [talvez exageraram na engenharia](https://www.youtube.com/watch?v=9yjZpBq1XBE) como as pessoas esperam na fila e reduzem seu tempo de espera. É também um problema complexo, mas bem estruturado, que pode se beneficiar de simulações de Monte Carlo.


img

Neste exemplo, simularemos clientes entrando em um banco, supermercado, etc., onde serão atendidos individualmente. Para simplificar, teremos apenas um atendente para atender a cada cliente, embora você possa adaptar isso para considerar vários atendentes. O objetivo desta simulação é verificar se uma fila se formará e sairá do controle, e teoricamente poderíamos usar isso para prever o tempo médio de espera dos clientes.

Antes de prosseguir, pergunte-se: **Quais são as duas distribuições de probabilidade que fazem sentido para este problema? Nós as estudamos! Pense e siga em frente.**

|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

Se você refletir bastante sobre isso, lembre-se de que estudamos muitas distribuições de probabilidade. A distribuição normal pode fazer sentido para o tempo de atendimento ao cliente, assumindo que o tempo de atendimento segue uma distribuição normal. Mas e quanto ao fluxo de clientes na loja? A Distribuição de Poisson pode ser tentadora, pois prevê `k` clientes em um determinado intervalo de tempo. Mas esses clientes chegariam em blocos simultaneamente, quando, na verdade, chegam em momentos únicos dentro desse intervalo. Portanto, a distribuição exponencial fará mais sentido, pois modela o tempo decorrido entre a chegada de cada cliente.

Vamos construir a simulação abaixo usando a distribuição normal e a distribuição exponencial. Os clientes levarão em média 3 minutos para serem processados, com um desvio padrão de 0,5 minuto. Modelaremos 20 clientes, em média, chegando a cada hora, mas, para sermos consistentes em minutos, isso equivale a US$ 1/3 de um cliente a cada minuto. Executaremos a simulação para os primeiros 100 clientes.

Observe que isso é um pouco complexo, mas a ideia é usar essas duas distribuições para criar uma simulação "realista". Execute a simulação e observe sua saída antes de mergulhar no código em si.

In [ ]:
import numpy as np
from numpy.random import normal, exponential

np.random.seed(0)  # usar semente para obter simulações "aleatórias" idênticas

media_tempo_caixa = 3  # minutos
desvio_tempo_caixa = 0.5  # minutos
media_taxa_chegada = 20 / 60  # clientes por minuto
quantidade_clientes = 100

# tempo entre a chegada de um cliente e o próximo
tempos_entre_clientes = exponential(scale=1/media_taxa_chegada, size=quantidade_clientes+2)  # adicionar 2 para evitar erros de índice

# tempo de chegada dos clientes em minutos desde o início da simulação
tempos_chegada_clientes = np.cumsum(tempos_entre_clientes)

# tempo de atendimento de cada cliente no caixa
tempos_atendimento_clientes = normal(loc=media_tempo_caixa, scale=desvio_tempo_caixa, size=quantidade_clientes+2)  # adicionar 2 para evitar erros de índice

# iniciar no tempo 0, mas pular para a chegada do primeiro cliente, e controlar se há cliente sendo atendido
# e quais clientes estão esperando
tempo_atual = tempos_chegada_clientes[0]
clientes_esperando = []

indice_cliente_chegando = 0
indice_cliente_atendendo = 0
inicio_atendimento_cliente_atual = tempos_chegada_clientes[0]

# processar clientes até que todos tenham chegado
while indice_cliente_chegando < quantidade_clientes:

    # tempo de chegada do cliente sendo atendido
    tempo_chegada_cliente_atendendo = tempos_chegada_clientes[indice_cliente_atendendo]

    # tempo agendado para o término do atendimento do cliente atual
    tempo_fim_atendimento_cliente = inicio_atendimento_cliente_atual + \
                                    tempos_atendimento_clientes[indice_cliente_atendendo]

    # tempo da próxima chegada de cliente
    def proxima_chegada_cliente(): return tempos_chegada_clientes[indice_cliente_chegando+1]

    # VERIFICAR QUAL EVENTO OCORREU COMPARANDO OS TEMPOS
    proximo_tempo_evento = None

    # se for o primeiro cliente
    if tempo_atual == inicio_atendimento_cliente_atual:
        print(f"{tempo_atual}: CLIENTE {indice_cliente_chegando} CHEGOU, SEM FILA, ATENDIMENTO IMEDIATO")
        proximo_tempo_evento = np.min([tempo_fim_atendimento_cliente, proxima_chegada_cliente()])

    # se um cliente chega
    elif tempo_atual == proxima_chegada_cliente():
        indice_cliente_chegando += 1  # incrementar o índice de clientes que chegaram

        # se não há fila e o cliente chegando é o próximo a ser atendido
        if indice_cliente_atendendo == indice_cliente_chegando:
            inicio_atendimento_cliente_atual = tempo_atual
            tempo_fim_atendimento_cliente = inicio_atendimento_cliente_atual + tempos_atendimento_clientes[indice_cliente_atendendo]

            print(f"{tempo_atual}: CLIENTE {indice_cliente_chegando} CHEGOU, SEM FILA, ATENDIMENTO IMEDIATO")
        # caso contrário, o cliente entra na fila
        else:
            clientes_esperando.append(indice_cliente_chegando)
            print(f"{tempo_atual}: CLIENTE {indice_cliente_chegando} CHEGOU, ENTROU NA FILA {clientes_esperando}")

        # agendar próximo evento: término do atendimento atual ou chegada de novo cliente
        proximo_tempo_evento = np.min([tempo_fim_atendimento_cliente, proxima_chegada_cliente()])

    # se um cliente termina o atendimento
    elif tempo_atual == tempo_fim_atendimento_cliente:

        # se a fila não está vazia, atender o próximo cliente da fila
        if clientes_esperando:
            clientes_esperando.pop(0)
            print(f"{tempo_atual}: CLIENTE {indice_cliente_atendendo} TERMINOU, CLIENTE {indice_cliente_atendendo + 1} "
                  f"REMOVIDO DA FILA {clientes_esperando}")

            inicio_atendimento_cliente_atual = tempo_atual

            # próximo evento é o término desse novo atendimento ou a chegada de um novo cliente
            proximo_tempo_evento = np.min([
                inicio_atendimento_cliente_atual + tempos_atendimento_clientes[indice_cliente_atendendo + 1],
                proxima_chegada_cliente()
            ])
        else:
            # se a fila está vazia, esperar o próximo cliente
            print(f"{tempo_atual}: CLIENTE {indice_cliente_atendendo} TERMINOU, ESPERANDO PELO CLIENTE {indice_cliente_atendendo + 1}")
            proximo_tempo_evento = proxima_chegada_cliente()

        indice_cliente_atendendo += 1  # processar o próximo cliente

    # avançar para o próximo evento
    tempo_atual = proximo_tempo_evento


Ao executar a simulação acima, você verá que uma fila se forma irremediavelmente após um tempo suficiente. Isso deve indicar que outro caixa pode ser necessário para atender os clientes! Você também pode experimentar tempos de processamento mais curtos ou intervalos maiores entre os clientes, e descobrirá que há um equilíbrio ideal em algum momento em que o processamento acompanha a fila.

É claro que só podemos capturar uma parte limitada em uma simulação. O mundo real é caótico e em constante mudança, mesmo dentro do mesmo dia! As tarifas que presumimos em nosso modelo podem ser precisas para as primeiras horas úteis, mas então a multidão do almoço chega e o modelo subjacente muda completamente. Às vezes, um único cliente pode entrar e praticamente obstruir o caixa sobre seu dia, e isso se desvia do que nosso modelo pressupõe. O fato de uma simulação só poder capturar uma parte limitada da realidade é o que chamamos de **lacuna entre simulação e realidade**. Como disse um famoso estatístico: "todos os modelos estão errados, alguns são úteis". Alguns modelos, como simulações de voo aprovadas pela FAA, são muito úteis e impressionantes. Mas outros modelos como o que construímos podem nos dar insights, mas ainda são um pouco ingênuos em relação à natureza da transitoriedade.

## Exercício

O proprietário de um prédio deseja instalar um novo elevador, mas quer ter certeza de que o limite de peso atingido é uma ocorrência improvável. Uma simulação é executada para testar o número máximo de passageiros permitido em um elevador e com que frequência o peso máximo de 900 kg será excedido. Acredita-se que entre 1 e 15 passageiros tenham a mesma probabilidade de viajar de elevador, e o peso médio de um passageiro é de 78 kg e o desvio padrão é de 8 kg.

Complete o código abaixo para executar 1.000 simulações e determinar se uma permissão para 15 passageiros pode resultar em violações do limite de 900 kg.

In [ ]:
from numpy.random import normal, randint

media, desvio = 172, 18
quantidade_testes = 1000

limite_peso = 2100
maximo_passageiros = 15

infracoes = 0
for i in range(quantidade_testes):

    # gera 0 ou 1 para cada possível passageiro
    # 1 se o passageiro aparece, 0 se ele não aparece
    passageiros_presentes = randint(low=0, high=2, size=maximo_passageiros)

    # gera um peso aleatório para um passageiro
    pesos_passageiros = normal(?, ?, ?)

    # multiplica e soma os pesos dos passageiros
    peso_total = sum(passageiros_presentes * ?)

    if ? > limite_peso:
        infracoes += 1

print(f"{infracoes}/{quantidade_testes} infrações")

### RESPOSTA A BAIXO

|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

In [ ]:
from numpy.random import normal, randint

media, desvio = 172, 18
quantidade_testes = 1000

limite_peso = 2100
maximo_numero_passageiros = 15

infracoes = 0
for i in range(quantidade_testes):

    # gera 0 ou 1 para cada possível passageiro
    # 1 se aquele passageiro aparecer, 0 se não aparecer
    passageiros_presentes = randint(low=0, high=2, size=maximo_numero_passageiros)

    # gera um peso aleatório para cada passageiro
    pesos_passageiros = normal(media, desvio, maximo_numero_passageiros)

    # multiplica e soma os pesos dos passageiros
    peso_total = sum(passageiros_presentes * pesos_passageiros)

    if peso_total > limite_peso:
        infracoes += 1

print(f"{infracoes}/{quantidade_testes} infrações")